In [41]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

hp = pd.read_excel(
    "data/hp_retro_data.xls", sheet_name="Continouos Data (Final)", skiprows=2
)
ap = pd.read_excel(
    "data/ap_pro_data.xls", sheet_name="Physicians - Continuous", skiprows=2
)

#### HP Retro

In [45]:
df = hp
df

,NUMBER,SEX,TEMP,HEART_RATE,AGE,PREV_VISIT,HX_TRAUMA,COMPLAINT_SITE,HX_ILLNESS,GAIT_REPORTED,...,GAIT_OBSERVED,PAIN_ROM_HIP,OTHER_PAIN_SITE,PAIN_PALPATION,SWELLING,CURRENT_ILLNESS,PREV_PROBLEMS,WBC,ESR,TRIAGE
0,1,FEMALE,?,96,12.679452,?,NO,HIP,NO,NOT_WT_BEAR,...,?,YES,?,TENDER,NO,NO,NO,?,?,DISCHARGE
1,3,FEMALE,37.6,126,2.572603,?,NO,LEG,NO,?,...,?,YES,NO,?,?,NO,NO,?,?,DISCHARGE
2,4,MALE,35.9,?,15.569863,?,?,HIP,NO,?,...,LIMP,YES,NO,?,NO,YES,?,?,?,DISCHARGE
3,5,MALE,36.3,80,16.758904,?,YES,HIP,?,?,...,NORMAL,YES,?,NORMAL,NO,NO,?,?,?,DISCHARGE
4,7,FEMALE,?,88,15.186301,?,YES,HIP,?,?,...,NOT_WT_BEAR,?,LEG,?,NO,?,NO,?,?,DISCHARGE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,409,FEMALE,37.3,148,3.668493,YES,YES,HIP,YES,NOT_WT_BEAR,...,?,YES,NO,NORMAL,NO,NO,?,18.5,18,LAB_XRAY_BSCAN
408,410,MALE,?,?,2.454795,YES,NO,HIP,NO,LIMP,...,NORMAL,NO,NO,NORMAL,NO,NO,YES,12.3,24,XRAY
409,411,MALE,36.7,80,9.731507,YES,NO,HIP,?,LIMP,...,?,YES,NO,?,?,?,?,?,?,XRAY
410,412,MALE,?,72,13.095890,YES,YES,LEG,NO,?,...,LIMP,NO,NO,NORMAL,NO,NO,?,7.38,15,XRAY


In [ ]:
df = df.replace("?", None)
df = df.drop(columns=["NUMBER"])

In [35]:
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_info = pd.DataFrame(
    {"Missing Values": missing_values, "Percentage": missing_percentage}
)
print(
    missing_info[missing_info["Missing Values"] > 0].sort_values(
        "Percentage", ascending=False
    )
)

                 Missing Values  Percentage
HIP_REST                    339   82.281553
PREV_VISIT                  279   67.718447
ESR                         250   60.679612
WBC                         241   58.495146
SWELLING                    150   36.407767
PREV_PROBLEMS               122   29.611650
GAIT_OBSERVED               122   29.611650
PAIN_PALPATION              120   29.126214
OTHER_PAIN_SITE             119   28.883495
HX_TRAUMA                    94   22.815534
HIP_FLEXION                  81   19.660194
PAIN_ROM_HIP                 80   19.417476
TEMP                         80   19.417476
APPEARANCE                   79   19.174757
HIP_INT_ROT                  71   17.233010
GAIT_REPORTED                56   13.592233
CURRENT_ILLNESS              51   12.378641
HX_ILLNESS                   45   10.922330
HIP_ROM                      37    8.980583
HEART_RATE                   27    6.553398
DURATION                     11    2.669903
COMPLAINT_SITE                7 

In [ ]:
# TODO:
# if percentage < 10 -> delete rows
# if percentage more than 70 -> delete column
# in other cases ???

In [39]:
df.columns.size

26

In [49]:
numeric_features = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = df.select_dtypes(include=["object", "category"]).columns.tolist()

target_variable = "Triage"
if target_variable in numeric_features:
    numeric_features.remove(target_variable)
if target_variable in categorical_features:
    categorical_features.remove(target_variable)

print(f"Numeric features: {len(numeric_features)}")
print(f"Categorical features: {len(categorical_features)}")

Numeric features: 1
Categorical features: 24
